# [pandas is cool! LB: 0.63714](https://www.kaggle.com/clustifier/pandas-is-cool-lb-0-63714/code)

#### Questions: 
``` 
ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 12*M) / (12 + ad_likelihood['count'])
```
#### Answers:
```
1. Regularization
2. When Ads show a lot of times, 'likelihood' tends to `ad_likelihood['sum'] / ad_likelihood['count']`
3. When Ads show few times, 'likelihood' tends to mean of all ads;
```

In [2]:
%%bash
head ../../data/unzip_data/clicks_train.csv
echo ""
head ../../data/unzip_data/clicks_test.csv

display_id,ad_id,clicked
1,42337,0
1,139684,0
1,144739,1
1,156824,0
1,279295,0
1,296965,0
2,125211,0
2,156535,0
2,169564,0

display_id,ad_id
16874594,66758
16874594,150083
16874594,162754
16874594,170392
16874594,172888
16874594,180797
16874595,8846
16874595,30609
16874595,143982


In [5]:
import numpy as np
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

logging.info("start")
dtypes={'ad_id':np.int32,"clicked":np.int8}
train=pd.read_csv("../../data/unzip_data/clicks_train.csv",usecols=['ad_id','clicked'],dtype=dtypes)
logging.info("read train done")
ad_likelihood=train.groupby('ad_id').clicked.agg(["mean","sum","count"]).reset_index()
M=train.clicked.mean()
L=12
ad_likelihood['likelihood']=(ad_likelihood["sum"]+L*M)/(ad_likelihood["count"]+L)
logging.info("get ad_likelihood done")

test=pd.read_csv("../../data/unzip_data/clicks_test.csv").astype(np.int32)
logging.info("read test done")
test=test.merge(ad_likelihood[['ad_id','likelihood']],on='ad_id',how='left').fillna(M)
logging.info("merge test & likelihood done")

test.sort_values(['display_id','likelihood'],inplace=True,ascending=False)
subm=test.groupby('display_id').ad_id.apply(lambda x:" ".join(map(str,x))).reset_index()
subm.to_csv("subm_pengju.csv",index=False)


2018-06-03 07:36:01,100 : INFO : get ad_likelihood done
2018-06-03 07:36:05,843 : INFO : read test done
2018-06-03 07:36:08,908 : INFO : merge test & likelihood done


In [6]:
%%bash
kaggle competitions submit -c outbrain-click-prediction -f subm_pengju.csv -m "init submission: pandas is cool! LB: 0.63714 by pengju change type of ad_id from np.float to np.int32; add log module;https://www.kaggle.com/clustifier/pandas-is-cool-lb-0-63714/code"

Successfully submitted to Outbrain Click Prediction

## Kernel same as Kaggle

In [ ]:
import pandas as pd
import numpy as np

dtypes = {'ad_id': np.float32, 'clicked': np.int8}

train = pd.read_csv("../../data/unzip_data/clicks_train.csv", usecols=['ad_id','clicked'], dtype=dtypes)

ad_likelihood = train.groupby('ad_id').clicked.agg(['count','sum','mean']).reset_index()
M = train.clicked.mean()
del train

ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 12*M) / (12 + ad_likelihood['count'])

test = pd.read_csv("../../data/unzip_data/clicks_test.csv")
test = test.merge(ad_likelihood, how='left')
test.likelihood.fillna(M, inplace=True)

test.sort_values(['display_id','likelihood'], inplace=True, ascending=False)
subm = test.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()
subm.to_csv("subm.csv", index=False)

In [ ]:
%%bash
kaggle competitions submit -c outbrain-click-prediction -f subm.csv -m "init submission: pandas is cool! LB: 0.63714"

## Summary

1. This method can be taken as one way of feature engineering to get parameter `likelihood`
2. It provides one useful way to handle mean value:
    - high frequency: tends to the original value
    - low frequency: tends to the global mean value
    - none : use the global mean value as the default 
3. We learn the command line method to commit results.     
4. We learn the groupby method to concat result using space.

In [ ]:
import matplotlib.pyplot as plt
ad_likelihood['count'].hist()
plt.show()

In [ ]:
test.head()

In [ ]:
subm.head()